In [31]:
import os
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json

# Ensure the file paths are correctly specified
secure_bundle_path = r'C:\Users\pramod\Desktop\INURON\Internship\Insurance_Premium_Prediction\research\secure-connect-insurance.zip'
token_path = r'C:\Users\pramod\Desktop\INURON\Internship\Insurance_Premium_Prediction\research\Insurance-token.json'

# Set environment variables in your Python environment if necessary
#os.environ["ASTRA_DB_SECURE_BUNDLE_PATH"] = secure_bundle_path
#os.environ["ASTRADB_APPLICATION_TOKEN"] = token_path

secure_bundle_path = os.environ["ASTRA_DB_SECURE_BUNDLE_PATH"]
token_path = os.environ["ASTRADB_APPLICATION_TOKEN"] 

# Cloud configuration for the Cassandra cluster
cloud_config = {
    'secure_connect_bundle': os.environ["ASTRA_DB_SECURE_BUNDLE_PATH"]
}

# Load the token JSON file
with open(os.environ["ASTRADB_APPLICATION_TOKEN"]) as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

# Set up authentication and connect to the cluster
auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

# Execute a query to test the connection
row = session.execute("SELECT release_version FROM system.local").one()
if row:
    print(row[0])
else:
    print("An error occurred.")


4.0.0.6816


In [2]:

# Set keyspace
session.set_keyspace('data_storage')

In [3]:
rows = session.execute("select * from insurance_records limit 5")
for row in rows:
    print(row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7])


bfed2eec-a04f-4e5c-93d8-956d1b9161d7 47 32.0 1 8551.349609375 southwest female no
43b7df5a-b4f2-4eee-b1d8-71c3ec76bb0f 18 24.100000381469727 1 2201.10009765625 southeast female no
d727516f-7b80-4b31-8479-306fcd073f09 50 27.399999618530273 0 25656.580078125 northeast female no
95c790c4-125b-464f-ac34-12312441e015 57 29.0 0 27218.439453125 northeast male yes
f3c1a491-3296-4736-b781-440c01493924 54 29.200000762939453 1 10436.099609375 southwest male no


In [5]:
import pandas as pd
df = pd.read_csv("insurance.csv")
df.head(3)

,age,sex,bmi,children,smoker,region,expenses
0,19,female,27.9,0,yes,southwest,16884.92
1,18,male,33.8,1,no,southeast,1725.55
2,28,male,33.0,3,no,southeast,4449.46


In [6]:
insurance_cols = []

rows = session.execute("select * from insurance_records")
for row in rows:
    insurance_cols.append([row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7]])

In [14]:
import pandas as pd

data = pd.DataFrame(insurance_cols, columns = ['id','age', "bmi",'children', 'expenses', 'region', 'sex', 'smoker'] )

In [18]:
data.head()

if "id" in data.columns :
    data.drop(columns = ['id'], inplace = True)

In [19]:
data.head()

,age,bmi,children,expenses,region,sex,smoker
0,47,32.000000,1,8551.349609,southwest,female,no
1,18,24.100000,1,2201.100098,southeast,female,no
2,50,27.400000,0,25656.580078,northeast,female,no
3,57,29.000000,0,27218.439453,northeast,male,yes
4,54,29.200001,1,10436.099609,southwest,male,no


In [24]:
data.drop_duplicates(inplace = True, ignore_index = True)

In [28]:
df.drop_duplicates(inplace = True, ignore_index = True)

In [29]:
data.shape, df.shape

((1337, 7), (1337, 7))

In [30]:
data.to_csv('insurance_from_db.csv',index=False)

In [ ]:
# logging folder creation check \

try :
    a = 10/"the"
except Exception as e :
    logging.info(e)
    raise InsuranceException(e, sys) from e 

pipline  = TrainPipeline()
pipline.run_pipeline()



In [ ]:
# astra db connection check 
# insurance prediction - configuaration - connection.py 


from Insurance_Prediction.configuration.connection import AstraDBClient
from Insurance_Prediction.exception import InsuranceException
from Insurance_Prediction.logger import logging
from Insurance_Prediction.constants import  DATA_INGESTION_KEYSPACE_NAME,ASTRA_DB_SECURE_BUNDLE_PATH

 
# Usage Example:
if __name__ == "__main__":
    try:
        db_client = AstraDBClient()

        # Example query execution
        result = db_client.execute_query("SELECT * FROM insurance_records LIMIT 5")
        if result:
            for row in result:
                print(row)
                logging.info(f"Row: {row}")
        else:
            print("No data found.")

    except InsuranceException as ie:
        print(f"Insurance Exception occurred: {ie}")

    finally:
        if db_client:
            db_client.close_connection()


In [ ]:
# exporting data from astra db and converting in to dataframe checks 
# # insurance prediction - data_access - insurance_data.py 


from Insurance_Prediction.logger import logging
from Insurance_Prediction.exception import InsuranceException
import sys 

from Insurance_Prediction.pipeline.training_pipeline import TrainPipeline

from Insurance_Prediction.data_access.insurance_data import InsuranceData

if __name__ == "__main__":
    try:
        insurance_data = InsuranceData()
        df = insurance_data.export_collection_as_dataframe("insurance_records")
        print(df.head())
    except InsuranceException as ie:
        print(f"Insurance Exception occurred: {ie}")


In [ ]:
# checking wheater the DataIngestion is creating artifact folder or not 

# # insurance prediction - components - data_ingestion_old.py 

import os
from Insurance_Prediction.entity.config_entity import DataIngestionConfig
from Insurance_Prediction.components.data_ingestion_old import DataIngestion
from Insurance_Prediction.exception import InsuranceException
import logging

def main():
    try:
        logging.basicConfig(level=logging.INFO)

        # Create a DataIngestionConfig instance with default or customized settings
        data_ingestion_config = DataIngestionConfig()

        # Initialize DataIngestion instance with the config
        data_ingestion = DataIngestion(data_ingestion_config=data_ingestion_config)

        # Call the export_data_into_feature_store method to create artifact folder and save CSV file
        dataframe = data_ingestion.export_data_into_feature_store()

        # Verify if the CSV file is created and the dataframe is not empty
        if os.path.exists(data_ingestion_config.feature_store_file_path):
            logging.info(f"CSV file created successfully at {data_ingestion_config.feature_store_file_path}")
        else:
            logging.error(f"CSV file was not created at {data_ingestion_config.feature_store_file_path}")

        if not dataframe.empty:
            logging.info("DataFrame is not empty, data ingestion successful.")
        else:
            logging.error("DataFrame is empty, data ingestion failed.")

    except InsuranceException as e:
        logging.error(f"InsuranceException occurred: {e}")
    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")

if __name__ == "__main__":
    main()



In [ ]:
# checking wheater the DataIngestion is creating artifact folder and ingestion - train and test folder (split_train_test_function) 

# # insurance prediction - components - data_ingestion_old.py 


import os
import sys
import logging
from Insurance_Prediction.entity.config_entity import DataIngestionConfig
from Insurance_Prediction.components.data_ingestion_old import DataIngestion
from Insurance_Prediction.exception import InsuranceException
import pandas as pd

def main():
    try:
        logging.basicConfig(level=logging.INFO)

        # Create a DataIngestionConfig instance with default or customized settings
        data_ingestion_config = DataIngestionConfig()

        # Initialize DataIngestion instance with the config
        data_ingestion = DataIngestion(data_ingestion_config=data_ingestion_config)

        # Call the export_data_into_feature_store method to create artifact folder and save CSV file
        dataframe = data_ingestion.export_data_into_feature_store()

        # Verify if the CSV file is created and the dataframe is not empty
        if os.path.exists(data_ingestion_config.feature_store_file_path):
            logging.info(f"CSV file created successfully at {data_ingestion_config.feature_store_file_path}")
        else:
            logging.error(f"CSV file was not created at {data_ingestion_config.feature_store_file_path}")

        if not dataframe.empty:
            logging.info("DataFrame is not empty, data ingestion successful.")
            
            # Call split_data_as_train_test method
            data_ingestion.split_data_as_train_test(dataframe)

            # Verify if the training and testing CSV files are created
            if os.path.exists(data_ingestion_config.training_file_path):
                logging.info(f"Training CSV file created successfully at {data_ingestion_config.training_file_path}")
            else:
                logging.error(f"Training CSV file was not created at {data_ingestion_config.training_file_path}")

            if os.path.exists(data_ingestion_config.testing_file_path):
                logging.info(f"Testing CSV file created successfully at {data_ingestion_config.testing_file_path}")
            else:
                logging.error(f"Testing CSV file was not created at {data_ingestion_config.testing_file_path}")
        else:
            logging.error("DataFrame is empty, data ingestion failed.")

    except InsuranceException as e:
        logging.error(f"InsuranceException occurred: {e}")
    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")

if __name__ == "__main__":
    main()



In [ ]:
# check the fuction initiate_data_ingestion created in the dataingestion class - which runs both artifact 
# export_data_into_feature_store , split_data_as_train_test


import logging
from Insurance_Prediction.entity.config_entity import DataIngestionConfig
from Insurance_Prediction.data_access.insurance_data import InsuranceData
from Insurance_Prediction.components.data_ingestion_old import DataIngestion
from Insurance_Prediction.logger import logging

# Configure logging
logging.basicConfig(level=logging.INFO)

def main():
    try:
        logging.info("Starting data ingestion process")
        
        # Initialize data ingestion config
        data_ingestion_config = DataIngestionConfig()

        # Initialize data ingestion object
        data_ingestion = DataIngestion(data_ingestion_config)

        # Initiate data ingestion
        data_ingestion_artifact = data_ingestion.initiate_data_ingestion()

        logging.info(f"Data ingestion completed successfully. Artifacts: {data_ingestion_artifact}")
        
    except Exception as e:
        logging.error(f"An error occurred: {e}")
        raise

if __name__ == "__main__":
    main()


In [ ]:
# checking traing pipe line - start data ingestion created in the run_pipeline

# it strat from the begging lke astradb connection, then exporting csv, spliting, storing data in the artifact and 
# train test folder


from Insurance_Prediction.pipeline.training_pipeline import TrainPipeline

pipeline = TrainPipeline()
pipeline.run_pipeline()



In [32]:
pwd

'C:\\Users\\pramod\\Desktop\\INURON\\Internship\\Insurance_Premium_Prediction\\research'